In [10]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier


In [11]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [12]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 631 entries, Unnamed: 0 to nk_feat_89
dtypes: float64(630), int64(1)
memory usage: 24.6 MB


In [13]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

In [14]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(objective= 'MultiClass',iterations=300, random_state=0,eval_metric='TotalF1:average=Micro')

In [16]:
#cross validation score
scores = cross_val_score(cat, x_train, np.ravel(y_train), cv=10, scoring='f1_micro',n_jobs=-1)


Learning rate set to 0.236581
Learning rate set to 0.236581
Learning rate set to 0.236584
Learning rate set to 0.236581
Learning rate set to 0.236581
0:	learn: 0.6277959	total: 1.73s	remaining: 8m 38s
Learning rate set to 0.236581
1:	learn: 0.6327904	total: 2.84s	remaining: 7m 3s
Learning rate set to 0.236581
2:	learn: 0.6267101	total: 3.13s	remaining: 5m 10s
3:	learn: 0.6386536	total: 3.62s	remaining: 4m 27s
4:	learn: 0.6373507	total: 3.98s	remaining: 3m 54s
5:	learn: 0.6408252	total: 4.66s	remaining: 3m 48s
6:	learn: 0.6514658	total: 4.93s	remaining: 3m 26s
Learning rate set to 0.236581
7:	learn: 0.6603692	total: 5.5s	remaining: 3m 20s
8:	learn: 0.6692725	total: 5.84s	remaining: 3m 8s
0:	learn: 0.6083370	total: 6.19s	remaining: 30m 50s
Learning rate set to 0.236584
Learning rate set to 0.236584
0:	learn: 0.6097720	total: 7.08s	remaining: 35m 15s
9:	learn: 0.6718784	total: 6.93s	remaining: 3m 20s
1:	learn: 0.6074685	total: 7.45s	remaining: 18m 29s
10:	learn: 0.6720955	total: 7.64s	rem

In [19]:
print(scores.mean())
print(scores.std())
#0.6884819135273974 StandardScaler
#0.7000122309197652 / 0.012079469299786984 MinMaxScaler
#0.7092003271771038 / 0.01337305017983602 new preprocessing
#0.7203335218933462 / 0.019186188119658828 new new preprocessing


0.7203335218933462
0.019186188119658828


In [17]:
import pickle
filename = 'models/catboost_model.sav'
pickle.dump(cat, open(filename, 'wb'))


In [18]:
cat.fit(x_train, np.ravel(y_train))

y_test = np.ravel(cat.predict(x_test))
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
data_y_test
df_y_test = pd.DataFrame(data_y_test, columns=['id', 'y'])
df_y_test.to_csv('outputs/output_cat.csv', index=False)

Learning rate set to 0.237857
0:	learn: 0.6226304	total: 578ms	remaining: 2m 52s
1:	learn: 0.6220442	total: 1.05s	remaining: 2m 36s
2:	learn: 0.6325972	total: 1.57s	remaining: 2m 35s
3:	learn: 0.6384600	total: 2.11s	remaining: 2m 36s
4:	learn: 0.6443228	total: 2.66s	remaining: 2m 37s
5:	learn: 0.6533125	total: 3.18s	remaining: 2m 36s
6:	learn: 0.6585890	total: 3.68s	remaining: 2m 34s
7:	learn: 0.6667970	total: 4.18s	remaining: 2m 32s
8:	learn: 0.6714872	total: 4.67s	remaining: 2m 31s
9:	learn: 0.6732460	total: 5.16s	remaining: 2m 29s
10:	learn: 0.6794997	total: 5.66s	remaining: 2m 28s
11:	learn: 0.6879031	total: 6.18s	remaining: 2m 28s
12:	learn: 0.6898573	total: 6.72s	remaining: 2m 28s
13:	learn: 0.6931796	total: 7.25s	remaining: 2m 28s
14:	learn: 0.6992378	total: 7.73s	remaining: 2m 26s
15:	learn: 0.7027555	total: 8.26s	remaining: 2m 26s
16:	learn: 0.7084229	total: 8.74s	remaining: 2m 25s
17:	learn: 0.7138949	total: 9.23s	remaining: 2m 24s
18:	learn: 0.7164354	total: 9.69s	remaining: